In [1]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from shapely.geometry import Point
from shapely.ops import nearest_points
import numpy as np

## Notebook for finding number of schools in 2km radius

In [2]:
properties = pd.read_csv("../data/curated/input_df.csv")
properties

,URL,Rent_Price,Address,Bedrooms,Bathrooms,Parking,Property_Type,Latitude,Longitude,Date_Available,...,Secure Parking,Intercom,Heating,Ensuite,Balcony / Deck,Fully fenced,Days_From_Now,negative_class,positive_class,geometr_2km
0,https://www.domain.com.au/7-isabella-way-tarne...,500.0,"7 Isabella Way, Tarneit VIC 3029",4.0,2.0,2.0,House,-37.837929,144.653245,Later,...,0,0,0,0,0,0,8.0,0.0,1.0,POLYGON ((16104725.59725854 -4556555.819541395...
1,https://www.domain.com.au/1906-570-lygon-stree...,460.0,"1906/570 Lygon Street, Carlton VIC 3053",1.0,1.0,0.0,Apartment / Unit / Flat,-37.793559,144.969056,Available Now,...,0,0,0,0,0,0,0.0,0.0,1.0,POLYGON ((16139881.53922936 -4550303.587526619...
2,https://www.domain.com.au/5-modra-street-point...,550.0,"5 Modra Street, Point Cook VIC 3030",4.0,2.0,2.0,House,-37.905707,144.773112,Available Now,...,0,0,0,0,0,0,0.0,0.0,1.0,POLYGON ((16118069.09726561 -4566113.910546225...
3,https://www.domain.com.au/5-63-droop-street-fo...,560.0,"5/63 Droop street, Footscray VIC 3011",2.0,1.0,1.0,Apartment / Unit / Flat,-37.796075,144.895730,Available Now,...,0,0,0,0,0,0,0.0,1.0,0.0,POLYGON ((16131718.92624745 -4550657.968199254...
4,https://www.domain.com.au/104-193-buckley-stre...,530.0,"104/193 Buckley Street, Essendon VIC 3040",2.0,1.0,1.0,Apartment / Unit / Flat,-37.756341,144.909078,Available Now,...,0,0,0,0,0,0,0.0,0.0,1.0,"POLYGON ((16133204.81881056 -4545061.96294156,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12296,https://www.domain.com.au/21-hannah-street-ben...,155.0,"21 Hannah Street, Benalla VIC 3672",0.0,0.0,0.0,House,-36.543437,145.984015,Available Now,...,0,0,0,0,0,0,0.0,1.0,0.0,POLYGON ((16252866.24715346 -4375657.443938157...
12297,https://www.domain.com.au/34-yosemite-street-t...,530.0,"34 Yosemite Street, Truganina VIC 3029",4.0,2.0,2.0,House,-37.759074,144.696542,Available Now,...,0,0,0,0,0,0,0.0,0.0,1.0,POLYGON ((16109545.37498752 -4545446.804402602...
12298,https://www.domain.com.au/39-kenny-street-west...,550.0,"39 Kenny Street, Westmeadows VIC 3049",4.0,2.0,2.0,House,-37.676119,144.900169,Available Now,...,0,0,0,0,0,0,0.0,0.0,1.0,POLYGON ((16132213.04007123 -4533772.742955274...
12299,https://www.domain.com.au/311-1728-dandenong-r...,550.0,"311/1728 Dandenong Road, Clayton VIC 3168",2.0,1.0,0.0,Apartment / Unit / Flat,-37.913004,145.124261,Later,...,0,0,0,0,0,0,0.0,0.0,1.0,POLYGON ((16157158.83627013 -4567143.517090854...


In [3]:
properties["Sentiment"]

0        0.425000
1        0.160000
2        0.300000
3        0.143750
4        0.506250
           ...   
12296    0.000000
12297    0.299167
12298    0.391667
12299    0.027273
12300    0.397143
Name: Sentiment, Length: 12301, dtype: float64

In [4]:
schools = pd.read_csv("../data/landing/dv346-schoollocations2023.csv",  encoding='ISO-8859-1' )
schools

,Education_Sector,Entity_Type,School_No,School_Name,School_Type,School_Status,Address_Line_1,Address_Line_2,Address_Town,Address_State,...,Postal_Address_Line_1,Postal_Address_Line_2,Postal_Town,Postal_State,Postal_Postcode,Full_Phone_No,LGA_ID,LGA_Name,X,Y
0,Government,1,1,Alberton Primary School,Primary,O,21 Thomson Street,NaN,Alberton,VIC,...,21 Thomson Street,NaN,ALBERTON,VIC,3971,03 5183 2412,681,Wellington (S),146.66660,-38.61771
1,Government,1,3,Allansford and District Primary School,Primary,O,Frank Street,NaN,Allansford,VIC,...,Frank Street,NaN,ALLANSFORD,VIC,3277,03 5565 1382,673,Warrnambool (C),142.59039,-38.38628
2,Government,1,4,Avoca Primary School,Primary,O,118 Barnett Street,NaN,Avoca,VIC,...,P O Box 12,NaN,AVOCA,VIC,3467,03 5465 3176,599,Pyrenees (S),143.47565,-37.08450
3,Government,1,8,Avenel Primary School,Primary,O,40 Anderson Street,NaN,Avenel,VIC,...,40 Anderson Street,NaN,AVENEL,VIC,3664,03 5796 2264,643,Strathbogie (S),145.23472,-36.90137
4,Government,1,12,Warrandyte Primary School,Primary,O,5-11 Forbes Street,NaN,Warrandyte,VIC,...,5-11 Forbes Street,NaN,WARRANDYTE,VIC,3113,03 9844 3537,421,Manningham (C),145.21398,-37.74268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2297,Independent,2,2244,Plenty River College,Secondary,O,"Unit 5, 9 Danaher Drive",NaN,SOUTH MORANG,VIC,...,"Unit 5, 9 Danaher Drive",NaN,SOUTH MORANG,VIC,3752,03 9435 9060,707,Whittlesea (C),145.08148,-37.64875
2298,Catholic,2,2245,Holy Cross Catholic Primary School,Primary,O,2-14 Carlingford Rd,NaN,MICKLEHAM,VIC,...,2-14 Carlingford Rd,NaN,MICKLEHAM,VIC,3064,03 9216 4200,327,Hume (C),144.90520,-37.53046
2299,Independent,2,2246,Sidrah Gardens School,Primary,O,434-442 Belgrave-Hallam Road,NaN,NARRE WARREN NORTH,VIC,...,434-442 Belgrave-Hallam Road,NaN,NARRE WARREN NORTH,VIC,3804,03 9796 9704,161,Casey (C),145.31589,-37.97324
2300,Independent,2,2247,Mountain District Community College,Secondary,O,13-15 The Avenue,NaN,FERNTREE GULLY,VIC,...,13-15 The Avenue,NaN,FERNTREE GULLY,VIC,3156,03 9758 7859,367,Knox (C),145.29327,-37.88319


In [5]:
schools_gdf = gpd.GeoDataFrame(schools, 
                               geometry=gpd.points_from_xy(schools['X'], schools['Y']),
                               crs="EPSG:4326")

In [6]:
geom_2km_gdf = gpd.GeoDataFrame(properties, geometry=gpd.GeoSeries.from_wkt(properties['geometr_2km']), crs="EPSG:3857")

In [7]:
schools_gdf = schools_gdf.to_crs(epsg=3857)
geom_2km_gdf = geom_2km_gdf.to_crs(epsg=3857)

In [8]:
print(geom_2km_gdf.crs)

EPSG:3857


In [9]:
schools_within_2km = gpd.sjoin(schools_gdf, geom_2km_gdf, how="inner", predicate="within")
schools_count = schools_within_2km.groupby("URL").size()

In [10]:
schools_gdf

,Education_Sector,Entity_Type,School_No,School_Name,School_Type,School_Status,Address_Line_1,Address_Line_2,Address_Town,Address_State,...,Postal_Address_Line_2,Postal_Town,Postal_State,Postal_Postcode,Full_Phone_No,LGA_ID,LGA_Name,X,Y,geometry
0,Government,1,1,Alberton Primary School,Primary,O,21 Thomson Street,NaN,Alberton,VIC,...,NaN,ALBERTON,VIC,3971,03 5183 2412,681,Wellington (S),146.66660,-38.61771,POINT (16326851.228 -4667058.811)
1,Government,1,3,Allansford and District Primary School,Primary,O,Frank Street,NaN,Allansford,VIC,...,NaN,ALLANSFORD,VIC,3277,03 5565 1382,673,Warrnambool (C),142.59039,-38.38628,POINT (15873089.607 -4634138.859)
2,Government,1,4,Avoca Primary School,Primary,O,118 Barnett Street,NaN,Avoca,VIC,...,NaN,AVOCA,VIC,3467,03 5465 3176,599,Pyrenees (S),143.47565,-37.08450,POINT (15971636.299 -4450891.551)
3,Government,1,8,Avenel Primary School,Primary,O,40 Anderson Street,NaN,Avenel,VIC,...,NaN,AVENEL,VIC,3664,03 5796 2264,643,Strathbogie (S),145.23472,-36.90137,POINT (16167455.076 -4425367.939)
4,Government,1,12,Warrandyte Primary School,Primary,O,5-11 Forbes Street,NaN,Warrandyte,VIC,...,NaN,WARRANDYTE,VIC,3113,03 9844 3537,421,Manningham (C),145.21398,-37.74268,POINT (16165146.310 -4543138.630)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2297,Independent,2,2244,Plenty River College,Secondary,O,"Unit 5, 9 Danaher Drive",NaN,SOUTH MORANG,VIC,...,NaN,SOUTH MORANG,VIC,3752,03 9435 9060,707,Whittlesea (C),145.08148,-37.64875,POINT (16150396.477 -4529924.112)
2298,Catholic,2,2245,Holy Cross Catholic Primary School,Primary,O,2-14 Carlingford Rd,NaN,MICKLEHAM,VIC,...,NaN,MICKLEHAM,VIC,3064,03 9216 4200,327,Hume (C),144.90520,-37.53046,POINT (16130773.077 -4513306.265)
2299,Independent,2,2246,Sidrah Gardens School,Primary,O,434-442 Belgrave-Hallam Road,NaN,NARRE WARREN NORTH,VIC,...,NaN,NARRE WARREN NORTH,VIC,3804,03 9796 9704,161,Casey (C),145.31589,-37.97324,POINT (16176490.879 -4575646.212)
2300,Independent,2,2247,Mountain District Community College,Secondary,O,13-15 The Avenue,NaN,FERNTREE GULLY,VIC,...,NaN,FERNTREE GULLY,VIC,3156,03 9758 7859,367,Knox (C),145.29327,-37.88319,POINT (16173972.832 -4562937.594)


In [11]:
schools_count = pd.DataFrame(schools_count)
schools_count = schools_count.reset_index()
schools_count

,URL,0
0,https://www.domain.com.au/-20-kosa-avenue-suns...,3
1,https://www.domain.com.au/-65-rosslyn-street-w...,9
2,https://www.domain.com.au/-leased-3-yarra-stre...,10
3,https://www.domain.com.au/006-903-dandenong-ro...,7
4,https://www.domain.com.au/04-390-burwood-highw...,3
...,...,...
11835,https://www.domain.com.au/unit-7-115-shaftesbu...,6
11836,https://www.domain.com.au/unit-9-2724-fourteen...,2
11837,https://www.domain.com.au/wheelers-hill-vic-31...,5
11838,https://www.domain.com.au/x03-342-346-centre-r...,7


In [12]:
schools_count.columns = ["URL", "School_Counts"]

In [13]:
merged_df = pd.merge(properties, 
                     schools_count,  
                     on='URL', 
                     how='left')

merged_df

,URL,Rent_Price,Address,Bedrooms,Bathrooms,Parking,Property_Type,Latitude,Longitude,Date_Available,...,Intercom,Heating,Ensuite,Balcony / Deck,Fully fenced,Days_From_Now,negative_class,positive_class,geometr_2km,School_Counts
0,https://www.domain.com.au/7-isabella-way-tarne...,500.0,"7 Isabella Way, Tarneit VIC 3029",4.0,2.0,2.0,House,-37.837929,144.653245,Later,...,0,0,0,0,0,8.0,0.0,1.0,POLYGON ((16104725.59725854 -4556555.819541395...,2.0
1,https://www.domain.com.au/1906-570-lygon-stree...,460.0,"1906/570 Lygon Street, Carlton VIC 3053",1.0,1.0,0.0,Apartment / Unit / Flat,-37.793559,144.969056,Available Now,...,0,0,0,0,0,0.0,0.0,1.0,POLYGON ((16139881.53922936 -4550303.587526619...,10.0
2,https://www.domain.com.au/5-modra-street-point...,550.0,"5 Modra Street, Point Cook VIC 3030",4.0,2.0,2.0,House,-37.905707,144.773112,Available Now,...,0,0,0,0,0,0.0,0.0,1.0,POLYGON ((16118069.09726561 -4566113.910546225...,3.0
3,https://www.domain.com.au/5-63-droop-street-fo...,560.0,"5/63 Droop street, Footscray VIC 3011",2.0,1.0,1.0,Apartment / Unit / Flat,-37.796075,144.895730,Available Now,...,0,0,0,0,0,0.0,1.0,0.0,POLYGON ((16131718.92624745 -4550657.968199254...,6.0
4,https://www.domain.com.au/104-193-buckley-stre...,530.0,"104/193 Buckley Street, Essendon VIC 3040",2.0,1.0,1.0,Apartment / Unit / Flat,-37.756341,144.909078,Available Now,...,0,0,0,0,0,0.0,0.0,1.0,"POLYGON ((16133204.81881056 -4545061.96294156,...",11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12296,https://www.domain.com.au/21-hannah-street-ben...,155.0,"21 Hannah Street, Benalla VIC 3672",0.0,0.0,0.0,House,-36.543437,145.984015,Available Now,...,0,0,0,0,0,0.0,1.0,0.0,POLYGON ((16252866.24715346 -4375657.443938157...,3.0
12297,https://www.domain.com.au/34-yosemite-street-t...,530.0,"34 Yosemite Street, Truganina VIC 3029",4.0,2.0,2.0,House,-37.759074,144.696542,Available Now,...,0,0,0,0,0,0.0,0.0,1.0,POLYGON ((16109545.37498752 -4545446.804402602...,NaN
12298,https://www.domain.com.au/39-kenny-street-west...,550.0,"39 Kenny Street, Westmeadows VIC 3049",4.0,2.0,2.0,House,-37.676119,144.900169,Available Now,...,0,0,0,0,0,0.0,0.0,1.0,POLYGON ((16132213.04007123 -4533772.742955274...,5.0
12299,https://www.domain.com.au/311-1728-dandenong-r...,550.0,"311/1728 Dandenong Road, Clayton VIC 3168",2.0,1.0,0.0,Apartment / Unit / Flat,-37.913004,145.124261,Later,...,0,0,0,0,0,0.0,0.0,1.0,POLYGON ((16157158.83627013 -4567143.517090854...,4.0


In [14]:
merged_df['School_Counts'] = merged_df['School_Counts'].fillna(0)
merged_df['School_Counts'].unique()

array([ 2., 10.,  3.,  6., 11., 15., 13.,  5.,  8.,  7.,  0., 12.,  9.,
        4.,  1., 14., 17., 16.])

In [15]:
merged_df['Rent_Price'] = pd.to_numeric(merged_df['Rent_Price'], errors='coerce')
merged_df = merged_df.dropna(subset=['Rent_Price', 'School_Counts'])

average_rent_by_school_count = merged_df.groupby('School_Counts')['Rent_Price'].mean().reset_index()

# Line plot to show changes over increasing number of schools
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=average_rent_by_school_count['School_Counts'],
    y=average_rent_by_school_count['Rent_Price'],
    mode='lines+markers',
    name='Average Rent Price',
    line=dict(color='blue', width=3),
    marker=dict(size=8)
))

fig.update_layout(
    title='Average Rent Price vs Number of Schools within 2km',
    xaxis_title="Number of Schools within 2km",
    yaxis_title="Average Rent Price ($)",
    template="plotly_white",
    height=600,
    width=900
)

fig.show()


In [16]:
pearson_corr, p_value_pearson = stats.pearsonr(merged_df['School_Counts'].astype(float), merged_df['Rent_Price'])

spearman_corr, p_value_spearman = stats.spearmanr(merged_df['School_Counts'].astype(float), merged_df['Rent_Price'])

model = ols('Rent_Price ~ C(School_Counts)', data=merged_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print(anova_table)
print(f"Pearson Correlation: {pearson_corr}, p-value: {p_value_pearson}")
print(f"Spearman Correlation: {spearman_corr}, p-value: {p_value_spearman}")

                        sum_sq       df          F        PR(>F)
C(School_Counts)  7.636719e+06     17.0  21.780303  3.700576e-67
Residual          2.533369e+08  12283.0        NaN           NaN
Pearson Correlation: 0.1241179397059041, p-value: 1.9860567882602615e-43
Spearman Correlation: 0.1286003126835906, p-value: 1.6155924759080553e-46


In [17]:
merged_df.to_csv("../data/curated/input.csv", index = False)

### Amenities

In [18]:
amenities= pd.read_csv("../data/landing/amenities.csv")
print(amenities['geometry'].dtype)
# amenities_gdf = gpd.GeoDataFrame(amenities, geometry='geometry')

object


In [19]:
import re

# Function to extract longitude and latitude from a POINT or POLYGON string
def extract_coordinates(geometry_str):
    if geometry_str.startswith("POINT"):
        match = re.findall(r'[-+]?\d*\.\d+|\d+', geometry_str)
        if len(match) == 2:
            longitude, latitude = float(match[0]), float(match[1])
            return longitude, latitude
    elif geometry_str.startswith("POLYGON"):
        match = re.findall(r'[-+]?\d*\.\d+|\d+', geometry_str)
        if len(match) >= 2:
            longitude, latitude = float(match[0]), float(match[1])
            return longitude, latitude
    return None, None

amenities['longitude'], amenities['latitude'] = zip(*amenities['geometry'].apply(lambda geom_str: extract_coordinates(geom_str) if isinstance(geom_str, str) else (None, None)))

amenities

,Unnamed: 0,tags,changeset,lat,version,timestamp,visible,id,lon,addr:city,...,geometry,osm_type,building:levels,landuse,parking,police,social_facility,poi_type,longitude,latitude
0,0,"{""emergency"":""no"",""healthcare"":""hospital""}",0.0,-37.884380,0,0,False,27288796,145.14557,NaN,...,POINT (145.1455688476562 -37.88438034057617),node,NaN,NaN,NaN,NaN,NaN,hospital,145.145569,-37.884380
1,1,"{""postal_code"":""3109"",""wikidata"":""Q28927045""}",0.0,-37.788254,0,0,False,89090066,145.15787,NaN,...,POINT (145.1578674316406 -37.78825378417969),node,NaN,NaN,NaN,NaN,NaN,police,145.157867,-37.788254
2,2,"{""brand"":""Coles"",""brand:wikidata"":""Q1108172"",""...",0.0,-37.791435,0,0,False,218028470,145.17184,NaN,...,POINT (145.1718444824219 -37.79143524169922),node,NaN,NaN,NaN,NaN,NaN,supermarket,145.171844,-37.791435
3,3,"{""branch"":""Carlton North"",""brand"":""Foodworks"",...",0.0,-37.788986,0,0,False,222420863,144.97609,NaN,...,POINT (144.9760894775391 -37.78898620605469),node,NaN,NaN,NaN,NaN,NaN,supermarket,144.976089,-37.788986
4,4,"{""addr:state"":""VIC"",""addr:suburb"":""Burwood"",""a...",0.0,-37.853460,0,0,False,256218806,145.09816,NaN,...,POINT (145.0981597900391 -37.85345840454102),node,NaN,NaN,NaN,NaN,NaN,fire_station,145.098160,-37.853458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1851,1851,"{""layer"":""-1"",""operator:wikidata"":""Q1406658"",""...",0.0,NaN,0,0,NaN,91353657031,NaN,NaN,...,POLYGON ((144.7407836914062 -37.73062133789062...,relation,NaN,NaN,NaN,NaN,NaN,police,144.740784,-37.730621
1852,1852,"{""operator:wikidata"":""Q1406658"",""operator:wiki...",0.0,NaN,0,0,NaN,95698207731,NaN,NaN,...,POLYGON ((144.8315277099609 -37.77803802490234...,relation,NaN,NaN,NaN,NaN,NaN,police,144.831528,-37.778038
1853,1853,"{""healthcare"":""hospital"",""operator:type"":""priv...",0.0,NaN,0,0,NaN,97986651576,NaN,NaN,...,POLYGON ((144.9842834472656 -37.81066131591797...,relation,NaN,NaN,NaN,NaN,NaN,hospital,144.984283,-37.810661
1854,1854,"{""operator:wikidata"":""Q1406658"",""type"":""multip...",0.0,NaN,0,0,NaN,3396403911,NaN,NaN,...,POLYGON ((145.0923004150391 -37.60089492797852...,relation,NaN,NaN,NaN,NaN,NaN,police,145.092300,-37.600895


In [20]:
amenities['geometry'] = amenities.apply(
    lambda row: Point(row['longitude'], row['latitude']) if pd.notnull(row['longitude']) and pd.notnull(row['latitude']) else None,
    axis=1
)

amenities = amenities[amenities['geometry'].notna()]

amenities_gdf = gpd.GeoDataFrame(amenities, geometry='geometry')

amenities_gdf.set_crs(epsg=4326, inplace=True)
print(amenities_gdf.head())

   Unnamed: 0                                               tags  changeset  \
0           0         {"emergency":"no","healthcare":"hospital"}        0.0   
1           1      {"postal_code":"3109","wikidata":"Q28927045"}        0.0   
2           2  {"brand":"Coles","brand:wikidata":"Q1108172","...        0.0   
3           3  {"branch":"Carlton North","brand":"Foodworks",...        0.0   
4           4  {"addr:state":"VIC","addr:suburb":"Burwood","a...        0.0   

         lat  version  timestamp visible         id        lon addr:city  ...  \
0 -37.884380        0          0   False   27288796  145.14557       NaN  ...   
1 -37.788254        0          0   False   89090066  145.15787       NaN  ...   
2 -37.791435        0          0   False  218028470  145.17184       NaN  ...   
3 -37.788986        0          0   False  222420863  144.97609       NaN  ...   
4 -37.853460        0          0   False  256218806  145.09816       NaN  ...   

                      geometry osm_typ

In [21]:
emergency_geometry = amenities.loc[amenities["poi_type"].isin(["police", "fire_station"]), 'geometry']
mall_geometry = amenities.loc[amenities["poi_type"].isin(["mall", "supermarket", "cafe"]), 'geometry']
medical_geometry = amenities.loc[amenities["poi_type"].isin(["hospital", "pharmacy"]), 'geometry']

medical = gpd.GeoDataFrame(geometry=medical_geometry)
emergency = gpd.GeoDataFrame(geometry=emergency_geometry)
mall = gpd.GeoDataFrame(geometry=mall_geometry)

medical.set_crs(epsg=3857, inplace=True)
emergency.set_crs(epsg=3857, inplace=True)
mall.set_crs(epsg=3857, inplace=True)

,geometry
2,POINT (145.172 -37.791)
3,POINT (144.976 -37.789)
5,POINT (145.183 -37.855)
6,POINT (145.240 -37.868)
7,POINT (145.164 -37.835)
...,...
1840,POINT (145.273 -37.882)
1841,POINT (145.273 -37.883)
1846,POINT (144.933 -37.610)
1849,POINT (144.884 -37.687)


In [22]:
medical_geometry

0       POINT (145.1455688476562 -37.88438034057617)
9       POINT (145.1649475097656 -37.85261535644531)
10      POINT (145.2859954833984 -37.86237335205078)
17       POINT (144.9440155029297 -37.7642936706543)
23      POINT (144.9949798583984 -37.91466522216797)
                            ...                     
1844    POINT (145.1200561523438 -37.92067718505859)
1845    POINT (144.9399719238281 -37.78859329223633)
1848    POINT (144.9474945068359 -37.72304534912109)
1850     POINT (144.9839172363281 -37.8117561340332)
1853    POINT (144.9842834472656 -37.81066131591797)
Name: geometry, Length: 659, dtype: object

In [23]:
medical['longitude'] = medical.geometry.x
medical['latitude'] = medical.geometry.y


properties_gdf = gpd.GeoDataFrame(properties, geometry=gpd.points_from_xy(properties['Longitude'], properties['Latitude']), crs="EPSG:4326")
properties_gdf['longitude'] = properties_gdf.geometry.x
properties_gdf['latitude'] = properties_gdf.geometry.y

fig = go.Figure()


fig.add_trace(go.Scattermapbox(
    lat=properties_gdf['latitude'],
    lon=properties_gdf['longitude'],
    mode='markers',
    marker=go.scattermapbox.Marker(
        size=9,
        color='blue',
        opacity=0.2

    ),
    text=properties_gdf['Address'], 
    name='Rental Properties'
))

fig.update_layout(
    mapbox_style="open-street-map",
    mapbox_zoom=10,
    mapbox_center={"lat": properties_gdf['latitude'].mean(), "lon": properties_gdf['longitude'].mean()},
    title="Rental Properties",
    legend=dict(y=0.99, x=0.01),  
    margin={"r":0,"t":0,"l":0,"b":0}
)

fig.show()


In [24]:
fig = px.box(
    properties,
    x='Dominant_Topic_Label',
    y='Rent_Price',
    color='Dominant_Topic_Label',
    title='Rental Price Distribution by Dominant Topic Label',
    labels={'Rent_Price': 'Rental Price', 'Dominant_Topic_Label': 'Dominant Topic Label'},
    template='plotly_white',
    color_discrete_sequence=px.colors.qualitative.Pastel # Optional: Use a distinct color palette
)

fig.update_layout(
    title={
        'text': 'Rental Price Distribution for Each Dominant Topic Label',
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='Dominant Topic Label',
    yaxis_title='Rental Price',
    showlegend=False
)

fig.show()

/Users/tinas/anaconda3/envs/garyenv/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [25]:
properties['Class'] = properties.apply(lambda row: 'Negative' if row['negative_class'] == 1 else 'Positive', axis=1)

# Plotting using Plotly
fig = px.box(
    properties,
    x='Class',
    y='Rent_Price',
    color='Class',
    title='Rental Price Distribution by Class',
    labels={'Rent_Price': 'Rental Price', 'Class': 'Class'},
    template='plotly_white',
    color_discrete_map={'Negative': 'red', 'Positive': 'green'}  # Optional: Color the classes for distinction
)

fig.update_layout(
    title={
        'text': 'Rental Price Distribution for Negative and Positive Classes',
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='Class',
    yaxis_title='Rental Price',
    showlegend=False
)

fig.show()

/Users/tinas/anaconda3/envs/garyenv/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [26]:
from scipy.stats import linregress

fig = px.density_contour(
    properties, 
    x="Sentiment", 
    y="Rent_Price", 
    title="Density Contour with Trend Line: Sentiment vs Rent Price", 
    labels={'Sentiment': 'Sentiment', 'Rent_Price': 'Rent Price (AUD)'},
    template='plotly_white'
)

# Update the trace to fill the contour regions for better visibility
fig.update_traces(contours_coloring="fill", colorscale="thermal")

# Calculate the linear regression line using scipy's linregress
slope, intercept, _, _, _ = linregress(properties['Sentiment'], properties['Rent_Price'])
x_vals = np.array(properties['Sentiment'])
y_vals = intercept + slope * x_vals

# Add the trend line to the plot
fig.add_trace(
    go.Scatter(
        x=x_vals,
        y=y_vals,
        mode='lines',
        name='Trend Line',
        line=dict(color='red', width=2)
    )
)

# Show the plot
fig.show()